In [1]:
#!unzip  data.zip

In [2]:
import sys
import numpy as np
import cv2


import os
import scipy.misc
from scipy.optimize import least_squares
import math
from copy import deepcopy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sfm_utils import *



In [3]:
image_data_dir = '../data/statue/'
unit_test_camera_matrix = np.load('../data/unit_test_camera_matrix.npy')
unit_test_image_matches = np.load('../data/unit_test_image_matches.npy')
image_paths = [os.path.join(image_data_dir, 'images', x) for x in sorted(os.listdir('../data/statue/images')) if '.jpg' in x]

In [4]:
im0 = cv2.imread(image_paths[0])
im_height, im_width, _ = im0.shape

In [5]:
print(len(image_paths))

5


In [6]:

focal_length = 719.5459
matches_subset = np.load(os.path.join(image_data_dir,'matches_subset.npy'), allow_pickle=True,encoding='latin1')[0,:]
dense_matches = np.load(os.path.join(image_data_dir, 'dense_matches.npy'),allow_pickle=True,encoding='latin1')
fundamental_matrices = np.load(os.path.join(image_data_dir,'fundamental_matrices.npy'),allow_pickle=True,encoding='latin1')[0,:]


In [7]:
print(matches_subset.shape)

(4,)


In [8]:
print(matches_subset[0].shape)

(4, 50)


In [9]:
for i in range(len(matches_subset)):
  print(matches_subset[i].shape)

(4, 50)
(4, 53)
(4, 63)
(4, 32)


In [10]:
print(dense_matches.shape)

(4,)


La représentation angle-axe, aussi appelée représentation axe-angle, est une façon de représenter une rotation en 3D en utilisant un axe de rotation et un angle de rotation autour de cet axe. C'est une représentation souvent plus intuitive qu'une matrice de rotation dans certains cas.

Pour convertir une matrice de rotation en représentation angle-axe, vous pouvez suivre ces étapes :

1. **Trouver l'axe de rotation** :

   L'axe de rotation peut être extrait directement de la matrice de rotation. Si $ R $ est votre matrice de rotation, l'axe de rotation $ \mathbf{n} $ peut être calculé comme suit :

   $$\mathbf{n} = \frac{1}{2\sin(\theta)} \begin{bmatrix} r_{32} - r_{23} \\ r_{13} - r_{31} \\ r_{21} - r_{12} \end{bmatrix} $$

   Où $ r_{ij} $ représente les éléments de la matrice de rotation.

2. **Calculer l'angle de rotation** :

   L'angle $ \theta $ peut être calculé comme :

   $$\theta = \arccos\left(\frac{\text{trace}(R) - 1}{2}\right) $$

   où $\text{trace}(R)$ est la somme des éléments diagonaux de la matrice de rotation.

3. **Normaliser l'axe** :

   Il est important de s'assurer que l'axe $ \mathbf{n} $ soit un vecteur unitaire. Vous pouvez le faire en divisant chaque composante de $ \mathbf{n} $ par sa magnitude :

   $$\mathbf{n} = \frac{\mathbf{n}}{\|\mathbf{n}\|} $$

La représentation résultante est $ (\mathbf{n}, \theta) $, où $ \mathbf{n} $ est le vecteur unitaire représentant l'axe de rotation, et $ \theta $ est l'angle de rotation autour de cet axe.



In [11]:
class Frame:
    def __init__(self, matches, focal_length, F, im_width, im_height):
        self.focal_length = focal_length
        self.im_height = im_height
        self.im_width = im_width
        self.matches = matches

        self.N = matches.shape[0]
        self.match_idx = np.array([np.arange(self.N), np.arange(self.N, 2 * self.N)])
        self.match_points = np.vstack((matches[:,:2], matches[:,2:]))

        self.K = np.eye(3)
        self.K[0,0] = self.K[1,1] = focal_length
        self.E = self.K.T.dot(F).dot(self.K)
        self.T = estimate_RT_from_E(self.E, matches.reshape((-1,2,2)), self.K)

        self.motion = np.zeros((2,3,4))
        self.motion[0,:,:-1] = np.eye(3)
        self.motion[1,:,:] = self.T
        self.structure = triangulate(self)

* Pour stocker les points et les correspondances, nous allons créer deux matrices :
* La première contenant tous les points des deux images
$$P=\begin{bmatrix}
p_{11}\\
p_{12}\\
\vdots\\
p_{1n}\\
p_{21}\\
p_{22}\\
\vdots\\
p_{2n}\\
\end{bmatrix}$$

* Est la correspondance avec les colonnes qui indiquent la caméra utilisée et la seconde est l'indice des points dans la première matrice.

$$\begin{bmatrix}
1 & n+1 \\
2 & n+2\\
\vdots & \\
n & 2*n
\end{bmatrix}$$

Frame 1 : Image 1 -> Image 2

Frame 2 : Image 2 -> Image 3

Correspandance : Image 1 -> Image 3

In [12]:
frames = [0] * (len(image_paths) - 1)
for i in range(len(image_paths)-1):
        frames[i] = Frame(matches_subset[i].T, focal_length,
                fundamental_matrices[i], im_width, im_height)
        bundle_adjustment(frames[i])

C:\Users\nmadali\Documents\Udemy-Reconstruction-3D\Chapitre 5\triangulation.py:197: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P_homo = np.array([P[0], P[1], P[2], 1.0])


In [13]:
for i in range(len(image_paths)-1):
  print(frames[i].match_idx.shape)

(2, 50)
(2, 53)
(2, 63)
(2, 32)


In [14]:
frameA=frames[0]
frameB=frames[1]

* Pour fusionner les points de correspondance entre les deux images, nous commençons par trouver les points qui sont similaires entre les images, puis nous ajoutons seulement les points de la troisième image à notre liste de points de correspondance, en les ajoutant en premier à la liste des points.

$$P=\left [ P, \begin{bmatrix}
p_{31}\\
p_{32}\\
\vdots \\
p_{3,c}
\end{bmatrix}\right ]$$

$$\begin{bmatrix}
1 & n+1& 2*n +1 \\
2 & n+2& -1\\
\vdots && \vdots \\
n & 2*n& 2*n +c
\end{bmatrix}$$

In [15]:
merged_frame = deepcopy(frameA)

In [16]:
print(frameA.match_idx.shape)

(2, 50)


In [17]:
trA = np.where(frameA.match_idx[0,:] >= 0)[0] # point cle sur la deuxieme image de la frame 1 donc la premiere  de la frame 1
xyA = frameA.match_points[frameA.match_idx[-1, trA], :]

In [18]:
trB = np.where(frameB.match_idx[0,:] >= 0)[0] # point cle sur la premier image de la frame 2 donc la deuxieme de la frame 1
xyB = frameB.match_points[frameB.match_idx[0, trB], :]

In [19]:
def row_intersection(A, B):
    nrows, ncols = A.shape
    dtype={'names':['f{}'.format(i) for i in range(ncols)],
                   'formats':ncols * [A.dtype]}
    intersect = np.intersect1d(A.view(dtype), B.view(dtype))
    intersect = intersect.view(A.dtype).reshape((-1,ncols))
    idA = np.array([np.where(np.all(A==x, axis=1))[0][0] for x in intersect])
    idB = np.array([np.where(np.all(B==x, axis=1))[0][0] for x in intersect])
    return intersect, idA, idB

In [20]:
xy_common, iA, iB = row_intersection(xyA, xyB)
xy_common = xy_common.T

In [21]:
#Ajoute une camera
merged_frame.match_idx = np.vstack((merged_frame.match_idx, neg_ones((1, merged_frame.match_idx.shape[1]))))

In [22]:
#0 -> 1


In [23]:
print(merged_frame.match_idx.shape)

(3, 50)


In [24]:
merged_frame.match_idx.shape

(3, 50)

In [25]:
merged_frame

In [26]:
length=1+1 #index de la nouvelle camera

In [28]:
# faire correspendre les points de la premier image de la frame 1 avec ceux de la deuxieme image de la frame 2
for i in range(xy_common.shape[1]):
        idA = trA[iA[i]]
        idB = trB[iB[i]]

        B_match_idx = frameB.match_idx[1, idB]
        # (camera 2, idx point in camera 1)=corespandance in camera 2
        merged_frame.match_points = np.vstack((merged_frame.match_points, frameB.match_points[B_match_idx, :]))
        merged_frame.match_idx[length, idA] = merged_frame.match_points.shape[0]-1


In [29]:
# L'une des caméras de l'image B est la même que celle de l'image A.
# Nous ajouterons tous les nouveaux points de cette caméra dans les champs de correspondance
xy_new, iB, iA = row_set_diff(xyB, xyA)
xy_new = xy_new.T

* Nous devons également ajouter les points qui ne correspondent pas entre les trois images, dans ce cas en ajoutant les points de départ et d'arrivée de la deuxième et de la troisième caméra.

In [32]:
def row_set_diff(A, B):
    nrows, ncols = A.shape
    dtype={'names':['f{}'.format(i) for i in range(ncols)],
                   'formats':ncols * [A.dtype]}
    set_diff = np.setdiff1d(A.view(dtype), B.view(dtype))
    set_diff = set_diff.view(A.dtype).reshape((-1,ncols))
    idA = []
    idB = []
    for x in set_diff:
        idx_in_A = np.where(np.all(A==x, axis=1))[0]
        idx_in_B = np.where(np.all(B==x, axis=1))[0]
        if len(idx_in_A) != 0:
            idA.append(idx_in_A[0])
        if len(idx_in_B) != 0:
            idB.append(idx_in_B[0])

    return set_diff, np.array(idA), np.array(idB)

In [34]:
for i in range(xy_new.shape[1]):
        idB = trB[iB[i]]

        merged_frame.match_points = np.vstack((merged_frame.match_points,frameB.match_points[frameB.match_idx[0,idB],:]))
        merged_frame.match_idx = np.hstack((merged_frame.match_idx, neg_ones((merged_frame.match_idx.shape[0],1))))
        merged_frame.match_idx[length-1,-1] = merged_frame.match_points.shape[0]-1

        merged_frame.structure = np.vstack((merged_frame.structure, frameB.structure[idB,:]))


        B_match_idx = frameB.match_idx[1, idB]
        merged_frame.match_points = np.vstack((merged_frame.match_points, frameB.match_points[B_match_idx,:]))
        merged_frame.match_idx[length,-1] = merged_frame.match_points.shape[0]-1




In [35]:
def merge_two_frames(frameA, frameB, length):
    merged_frame = deepcopy(frameA)

    frameB_to_A = multiply_transformations(inverse(frameA.motion[-1,:,:]), frameB.motion[0,:,:])
    frameB.structure = transform_points(frameB.structure, frameB_to_A)
    for i in range(2):
        frameB.motion[i,:,:] = multiply_transformations(frameB.motion[i,:,:], inverse(frameB_to_A))

    # puisque la caméra se trouve dans le cadre de référence fusionné, l'ajouter à la matrice de mouvement
    merged_frame.motion = np.vstack((merged_frame.motion, frameB.motion[-1,:,:].reshape((-1,3,4))))

    # nous devons réconcilier les points appariés pour générer la structure.
    # Nous devons fusionner les points appariés de chaque image supplémentaire, mais nous devons associer les points qui correspondent à des points déjà vus dans la même image.
    # associer les points qui correspondent à des points déjà vus dans la même
    # colonne
    trA = np.where(frameA.match_idx[0,:] >= 0)[0]
    xyA = frameA.match_points[frameA.match_idx[-1, trA], :]

    trB = np.where(frameB.match_idx[0,:] >= 0)[0]
    xyB = frameB.match_points[frameB.match_idx[0, trB], :]

    xy_common, iA, iB = row_intersection(xyA, xyB)
    xy_common = xy_common.T

    merged_frame.match_idx = np.vstack((merged_frame.match_idx, neg_ones((1, merged_frame.match_idx.shape[1]))))
    for i in range(xy_common.shape[1]):
        idA = trA[iA[i]]
        idB = trB[iB[i]]

        B_match_idx = frameB.match_idx[1, idB]

        merged_frame.match_points = np.vstack((merged_frame.match_points, frameB.match_points[B_match_idx, :]))
        merged_frame.match_idx[length, idA] = merged_frame.match_points.shape[0]-1

    # L'une des caméras de l'image B est la même que celle de l'image A.
    # Nous ajouterons tous les nouveaux points de cette caméra dans les champs de correspondance
    xy_new, iB, iA = row_set_diff(xyB, xyA)
    xy_new = xy_new.T

    for i in range(xy_new.shape[1]):
        idB = trB[iB[i]]

        merged_frame.match_points = np.vstack((merged_frame.match_points,frameB.match_points[frameB.match_idx[0,idB],:]))
        merged_frame.match_idx = np.hstack((merged_frame.match_idx, neg_ones((merged_frame.match_idx.shape[0],1))))
        merged_frame.match_idx[length-1,-1] = merged_frame.match_points.shape[0]-1
        merged_frame.structure = np.vstack((merged_frame.structure, frameB.structure[idB,:]))

        B_match_idx = frameB.match_idx[1, idB]

        merged_frame.match_points = np.vstack((merged_frame.match_points, frameB.match_points[B_match_idx,:]))
        merged_frame.match_idx[length,-1] = merged_frame.match_points.shape[0]-1



    return merged_frame


def merge_all_frames(frames):
    merged_frame = deepcopy(frames[0])
    for i in range(1,len(frames)):
        merged_frame = merge_two_frames(merged_frame, frames[i], i+1)
        merged_frame.structure = triangulate(merged_frame)
        bundle_adjustment(merged_frame)
        remove_outliers(merged_frame, 10)
        bundle_adjustment(merged_frame)

    return merged_frame


In [36]:
merged_frame = merge_all_frames(frames)